#  Multistage Compression Refrigeration Cycle

## 1 The Two-stage Compression Refrigeration Cycle

>Yunus A. Cengel, Michael A. Boles, Thermodynamics: An Engineering Approach, 8th Edition,McGraw-Hill, 2015
>
>EXAMPLE 11–4: Multistage Compression Refrigeration: Page627-628



**EXAMPLE 11–5 Two-Stage Refrigeration Cycle with a Flash Chamber,Page627-628**

Consider a two-stage compression refrigeration system operating between the pressure limits of 0.8 and 0.14 MPa. 

![two stage](./img/vcr/two_stage.jpg)

The working fluid is  R134a.

The refrigerant leaves the condenser as a saturated liquid and is throttled to a flash chamber operating at 0.32 MPa. 

Part of the refrigerant evaporates during this flashing process, and this vapor is mixed with the refrigerant leaving the low-pressure compressor. 

The mixture is then compressed to the condenser pressure by the high-pressure compressor. 

The liquid in the flash chamber is throttled to the evaporator pressure and cools the refrigerated space as it vaporizes in the evaporator. 

Assuming the refrigerant leaves the evaporator as a saturated vapor and both compressors are isentropic,

**Determine**

* (a) the fraction of the refrigerant that evaporates as it is throttled to the flash chamber,
* (b) the amount of heat removed from the refrigerated space and the compressor work per unit mass of refrigerant flowing through the condenser, and
* (c) the coefficient of performance.


## 2 The Analysis


### 2.1 The New Device Classes

* Flash Chamber

* Mixing Chamber

#### 2.1.1  Flash Chamber

In [ ]:
# %load ../../SimVCCE/playground/ncomponents/flashchamber.py

"""
class FlashChamber
                         ↓   iPort 
                   ┌─────┴─────┐
                   │           │
                   │           │→  oPortV
                   │────────── │
                   └─────┬─────┘
                         ↓  oPortL 
json example:
      {
            "name": "flashchamber",
            "devtype": "FLASH_CHAMBER",
            "iPort": {
                "p": 0.6
            },
            "oPortL": {
                "x": 0.0
            },
            "oPortV": {  "x": 1.0
            }
        },
     
"""
from components.port import Port

class FlashChamber:
    energy = "none"
    devtype = "FLASH_CHAMBER"

    def __init__(self, dictDev):
        """
        Initializes lashChamber
        """
        self.name = dictDev['name']
        self.iPort = Port(dictDev['iPort'])
        self.oPortV = Port(dictDev['oPortV'])
        self.oPortL = Port(dictDev['oPortL'])

    def state(self):
        if self.iPort.p is not None:
            self.oPortV.p = self.iPort.p
            self.oPortL.p = self.oPortL.p
            
        elif self.oPortL.p is not None:
            self.oPortV.p = self.oPortV.p
            self.iPort.p = self.iPort.p
       
        elif self.oPortV.p is not None:
            self.oPortL.p = self.oPortL.p
            self.iPort.p = self.iPort.p
      
    def balance(self):
        """flash chamber """
        oPortV_fdot =self.iPort.x
        oPortL_fdot =1.0-self.iPort.x
        self.oPortV.mdot = self.iPort.mdot*oPortV_fdot
        self.oPortL.mdot = self.iPort.mdot*oPortL_fdot
    
       
    def __str__(self):
        result = '\n' + self.name
        result += '\n' + " PORT " + Port.title
        result += '\n' + " iPort "+self.iPort.__str__()
        result += '\n' + " oPortV " + self.oPortV.__str__()
        result += '\n' + " oPortL " + self.oPortL.__str__()
        return result


#### 2.1.2 Mixing Chamber

In [ ]:
%load ../../SimVCCE/playground/ncomponents/mixingchamber.py
"""
MixingChamber
                         ↑ oPort 
                   ┌─────┴─────┐
                   │           │
                 → │           │ 
          iPort0   │           │
                   └─────┬─────┘
                         ↑ iPort1 
 
 json example:
  {
            "name": "mixingchamber",
            "devtype": "MIXING_CHAMBER",
            "iPort0": {
                "x": 1.0
            },
            "iPort1": {
            },
            "oPort": {
                "p": 0.6
            }
        }
"""

from components.port import Port


class MixingChamber:
    energy = "none"
    devtype = "MIXING_CHAMBER"

    def __init__(self, dictDev):
        """
        Initializes Merge_Two2one
        """
        self.name = dictDev['name']
        self.iPort0 = Port(dictDev['iPort0'])
        self.iPort1 = Port(dictDev['iPort1'])
        self.oPort = Port(dictDev['oPort'])

    def state(self):
        if self.iPort0.p is not None:
            self.oPort.p = self.iPort0.p
            self.iPort1.p = self.iPort0.p
        elif self.iPort1.p is not None:
            self.iPort0.p = self.iPort1.p
            self.oPort.p = self.iPort1.p
        elif self.oPort.p is not None:
            self.iPort0.p = self.oPort1.p
            self.iPort1.p = self.oPort1.p

    def balance(self):
        self.oPort.mdot = self.iPort0.mdot+self.iPort1.mdot
        self.oPort.h = (self.iPort0.mdot*self.iPort0.h +
                           self.iPort1.mdot*self.iPort1.h)/self.oPort.mdot

    def __str__(self):
        result = '\n' + self.name
        result += '\n' + " PORT " + Port.title
        result += '\n' + " iPort0 "+self.iPort0.__str__()
        result += '\n' + " iPort1 " + self.iPort1.__str__()
        result += '\n' + " oPort " + self.oPort.__str__()
        return result



### 2.2 The JOSN of Cycles


In [ ]:
# %load ../../SimVCCE/playground/jsonmodel/vcr_two_stage_11_5.json
{
    "name": "The_TWO_STAGE_11_5",
    "refrigerant": "R134a",
    "components": [
        {
            "name": "Compressor_HP",
            "devtype": "COMPRESSOR",
            "iPort": {
                "p": 0.32,
                "mdot": 1.0
            },
            "oPort": {
                "p": 0.8
            },
            "ef": 1.0
        },
        {
            "name": "Compressor_LP",
            "devtype": "COMPRESSOR",
            "iPort": {
                "p": 0.14
            },
            "oPort": {
                "p": 0.32
            },
            "ef": 1.0
        },
        {
            "name": "ExpansionValve1",
            "devtype": "EXPANSIONVALVE",
            "iPort": {
                "p": 0.8,
                "x": 0.0
            },
            "oPort": {
                "p": 0.32
            }
        },
        {
            "name": "ExpansionValve2",
            "devtype": "EXPANSIONVALVE",
            "iPort": {
                "p": 0.32
            },
            "oPort": {}
        },
        {
            "name": "Condenser",
            "devtype": "CONDENSER",
            "iPort": {
                "p": 0.8
            },
            "oPort": {
                "p": 0.8,
                "x": 0.0
            }
        },
        {
            "name": "flashchamber",
            "devtype": "FLASH_CHAMBER",
            "iPort": {
                "p": 0.32
            },
            "oPortL": {
                "p": 0.32,
                "x": 0.0
            },
            "oPortV": {
                "p": 0.32,
                "x": 1.0
            }
        },
        {
            "name": "Evaporator",
            "devtype": "EVAPORATOR",
            "iPort": {},
            "oPort": {
                "p": 0.14,
                "x": 1.0
            }
        },
        {
            "name": "Mixing_Chamber",
            "devtype": "MIXING_CHAMBER",
            "iPort0": {
                "p": 0.32,
                "x": 1
            },
            "iPort1": {
                "p": 0.32
            },
            "oPort": {
                "p": 0.32
            }
        }
    ],
    "connectors": [
        [
            {
                "devname": "Compressor_HP",
                "port": "oPort"
            },
            {
                "devname": "Condenser",
                "port": "iPort"
            }
        ],
        [
            {
                "devname": "Condenser",
                "port": "oPort"
            },
            {
                "devname": "ExpansionValve1",
                "port": "iPort"
            }
        ],
        [
            {
                "devname": "ExpansionValve1",
                "port": "oPort"
            },
            {
                "devname": "flashchamber",
                "port": "iPort"
            }
        ],
        [
            {
                "devname": "flashchamber",
                "port": "oPortV"
            },
            {
                "devname": "Mixing_Chamber",
                "port": "iPort0"
            }
        ],
        [
            {
                "devname": "flashchamber",
                "port": "oPortL"
            },
            {
                "devname": "ExpansionValve2",
                "port": "iPort"
            }
        ],
        [
            {
                "devname": "ExpansionValve2",
                "port": "oPort"
            },
            {
                "devname": "Evaporator",
                "port": "iPort"
            }
        ],
        [
            {
                "devname": "Evaporator",
                "port": "oPort"
            },
            {
                "devname": "Compressor_LP",
                "port": "iPort"
            }
        ],
        [
            {
                "devname": "Compressor_LP",
                "port": "oPort"
            },
            {
                "devname": "Mixing_Chamber",
                "port": "iPort1"
            }
        ],
        [
            {
                "devname": "Mixing_Chamber",
                "port": "oPort"
            },
            {
                "devname": "Compressor_HP",
                "port": "iPort"
            }
        ]
    ]
}

### 2.3 The Main Application

  


In [ ]:
# %load ../../SimVCCE/playground/demo_two_stage.py

"""
General Object-oriented Abstraction of VC Cycle 

EXAMPLE 11–5: Multistage Compression Refrigeration: Page627-628

Yunus A. Cengel, Michael A. Boles. Thermodynamics: An Engineering Approach, 8th Edition, McGraw-Hill,2015

Author: Cheng Maohua cmh@seu.edu.cn
"""
import sys
import os
curpath = os.path.abspath(os.path.dirname(__file__))
sys.path.append(curpath+'/../vccpython/')


from ncomponents.flashchamber import FlashChamber
from ncomponents.mixingchamber import MixingChamber

from components import compdict
from vcc.vccobj import VCCycle
from vcc.utils import OutFiles, create_dictcycle_from_jsonfile


if __name__ == "__main__":
    # add the newc omponents to compdict
    compdict[FlashChamber.devtype] = FlashChamber
    compdict[MixingChamber.devtype] = MixingChamber

    curpath = os.path.abspath(os.path.dirname(__file__))
    ResultFilePath = curpath+'/result/'

    json_filename = curpath+'\\'+'./jsonmodel/vcr_two_stage_11_5.json'

    thedictcycle = create_dictcycle_from_jsonfile(json_filename)
    cycle = VCCycle(thedictcycle)
    cycle.simulator()
    OutFiles(cycle)
    ResultFileName = ResultFilePath+thedictcycle['name']
    OutFiles(cycle, ResultFileName + '.txt')
